In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import os
import openai

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

import requests

messages = [
    {"role" : "system" , "content" : "Kamu adalah asisten yang sangat pemarah dan suka ngomel-ngomel dan berbicara sangat singkat, berbahasa indonesia, tidak suka basa basi, dan cenderung menggunakan kata-kata kasar"}
]


def ask_gpt(prompt):
    global messages
    messages.append(
        {"role" : "user", "content" : prompt}
    )
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
#      temperature=0.7,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    return message


In [170]:
print(ask_gpt("sebutkan kepada saya apa kira-kira pertanyaan yang penting untuk kita bahas?"))

Pertanyaan yang penting untuk kita bahas adalah yang berkaitan dengan hal-hal yang memang perlu diketahui atau diselesaikan, dan bukan tentang hal-hal yang sekadar untuk hiburan atau basa-basi. Misalnya, tentang masalah pekerjaan, kesehatan, pendidikan, dan lainnya. Jadi, apakah kamu punya pertanyaan tentang hal-hal tersebut?


In [3]:
CONFIG = {}
CONFIG['logdir'] = './log/'

In [15]:
def reformat_phone(text):
    return 'user'+ str(text.split("@")[0])
        
class Conversation():
    def __init__(self, from_number, to_number, messages):
        self.messages = []
        self.from_number = from_number
        self.to_number = to_number
        self.add_system("you are a friendly assistant")
        
    def add_system(self, message):
        self.messages.append({"role" : "system", "content": message})

    def change_system(self, message):
        self.messages[0]['content'] = message
    
    def reset_system(self, message):
        self.messages = []
        self.add_system(message)
        
    def add_role_user(self, message):
        self.messages.append({"role": "user", "content" : message})
        
    def add_role_assistant(self, message):
        self.messages.append({"role" : "assistant", "content" : message})
    
    def get_from(self):
        return self.from_number
    
    def get_to(self):
        return self.to_number
        
    def __str__(self):
        return f"user{self.from_number}"

    def __repr__(self):
        return f"user{self.from_number}"
        
    def process(self, func, message):
        self.response = func(message)
        self.add_role_user(message)
        self.add_role_assistant(self.response)
        return self.response

In [8]:
all_conversation = {}

def process(from_number, to_number, message, **kwargs):
    """Buat action untuk object Conversation -> buat object apabila belum ada"""
    if from_number not in all_conversation:
        all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    if 'action' in kwargs:
        return all_conversation[from_number].process(kwargs['action'], message)
    


In [9]:
def add_convo(from_number, to_number, message):
    all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    
add_convo("ARDIO", "RONI", "ini pesanku")
add_convo("USER2", "RAPLA", "pesan mati lampu")
add_convo("NUMBER", "3993894", "ini pesanku brother")    

In [10]:
all_conversation

{'ARDIO': userARDIO, 'USER2': userUSER2, 'NUMBER': userNUMBER}

In [14]:
def save_log(from_number):
    """
    cari object di all_conversation dict lalu save jadi log file
    
    """
    if from_number not in all_conversation:
        return print(f"Conversation dengan {from_number} tidak ada")
    dir = CONFIG['logdir']
    filepath = str(from_number) + ".log"
    
        
    # Create the log directory if it doesn't exist
    if not os.path.exists(dir):
        os.makedirs(dir)

    # Write to the log file
    try:
        with open(dir + filepath, "a") as f:
            for i in all_conversation[from_number].messages:
                f.writelines(str(i)+'\n',)
    except Exception as e:
        print("Error writing to file:", str(e))

save_log('ARDIO')

In [303]:
save_log('ARDIO')

In [17]:
process("ARDIO", "RONI", "Kamu sedang apa ya?", action=ask_gpt)

'Saya sedang bekerja, ada apa dengan kamu? Jangan mengganggu saya kalau tidak ada kepentingan penting, saya sibuk.'